# Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import dump, load
import time
from scipy.interpolate import PchipInterpolator
import scipy

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

from itertools import chain

# Data

In [2]:
df_train = pd.read_csv("../files/data/Surface_Crack/RN_TRAIN.csv", index_col=0)
df_train = df_train.drop(columns=['mgfw', 'mg', 'E', 'sigma', 'a'])
df_train = df_train.reindex(['a/c', 'a/t', 'c/b', 'phi', "KI"], axis=1)
d_train = df_train.to_numpy()
df_train, d_train.shape

(         a/c   a/t  c/b       phi         KI
 0       0.90  0.60  0.3  2.120567  34.597826
 1       0.75  0.65  0.3  0.853478  40.001090
 2       1.95  0.30  0.3  1.329954  14.470167
 3       0.90  0.40  0.3  2.979193  29.456790
 4       1.00  0.60  0.3  0.371790  36.227644
 ...      ...   ...  ...       ...        ...
 601662  0.65  0.50  0.3  2.445190  35.095373
 601663  1.80  0.25  0.3  0.037024  20.628363
 601664  0.30  0.65  0.4  0.007405  49.395277
 601665  1.15  0.60  0.3  1.324708  29.393773
 601666  0.55  0.80  0.2  0.413674  52.415671
 
 [601667 rows x 5 columns],
 (601667, 5))

In [3]:
# Getting all a/c, a/t and c/b combinations
SC_train_combinations = [d_train[0,:3]]

for line in d_train:
    if (line[:3] == SC_train_combinations[-1]).all():
        continue
    else:
        SC_train_combinations.append(line[:3])

SC_train_combinations = np.unique(SC_train_combinations, axis=0)

len(SC_train_combinations)

2518

In [4]:
print("Different a/c values: ", np.unique(SC_train_combinations[:,0], axis=0))
print("Different a/t values: ", np.unique(SC_train_combinations[:,1], axis=0))
print("Different c/b values: ", np.unique(SC_train_combinations[:,2], axis=0))

Different a/c values:  [0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75 0.8  0.85
 0.9  0.95 1.   1.05 1.1  1.15 1.2  1.25 1.3  1.35 1.4  1.45 1.5  1.55
 1.6  1.65 1.7  1.75 1.8  1.85 1.9  1.95 2.  ]
Different a/t values:  [0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75 0.8  0.85]
Different c/b values:  [0.01 0.1  0.2  0.3  0.4 ]


In [5]:
min_phi = []
max_phi = []

for (i,combination) in enumerate(SC_train_combinations):
    indices = np.where((d_train[:, 0] == combination[0]) & 
                    (d_train[:, 1] == combination[1]) &
                    (d_train[:, 2] == combination[2])) 
    indices = indices[0]
    phis = d_train[indices][:,-2]

    min_phi.append(phis.min())
    max_phi.append(phis.max())

min_phi = np.max(np.unique(min_phi))
max_phi = np.min(np.unique(max_phi))

SC_train_phi_regular_64 = np.linspace(min_phi, max_phi, 64)
SC_train_K_regular_64 = np.zeros((len(SC_train_combinations), len(SC_train_phi_regular_64)))
for (i,combination) in enumerate(SC_train_combinations):
    indices = np.where((d_train[:, 0] == combination[0]) & 
                    (d_train[:, 1] == combination[1]) &
                    (d_train[:, 2] == combination[2])) 
    indices = indices[0]

    phi_vals = d_train[indices][:,-2]
    K_vals = d_train[indices][:,-1]
    phi_idxes = np.argsort(phi_vals)
    
    monotonic_phi_idxes = [phi_idxes[0]]
    prev_phi = phi_vals[phi_idxes[0]]
    for index in phi_idxes[1:]:
        now_phi = phi_vals[index]
        if now_phi > prev_phi:
            monotonic_phi_idxes.append(index)
            prev_phi = phi_vals[index]
    
    interp_func = PchipInterpolator(phi_vals[monotonic_phi_idxes], K_vals[monotonic_phi_idxes], extrapolate=False)

    SC_train_K_regular_64[i,:] = interp_func(SC_train_phi_regular_64)

In [6]:
np.min(SC_train_K_regular_64), np.max(SC_train_K_regular_64)

(np.float64(10.854542515306234), np.float64(90.69109971391299))